In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [8]:
# creating dataframe from file
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Showing Dataframe
home_sales_df.show(3)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
only showing top 3 rows



In [10]:
# Create a temporary view of the data frame
home_sales_df.createOrReplaceTempView('home_sales')
home_sales_df.printSchema()


root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [15]:
# Importing relevant libraries
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import year, to_date, regexp_replace

# Converting data types from string to integer for mathematical manipulations
home_sales_df = home_sales_df.withColumn("price",home_sales_df["price"].cast(IntegerType()))
home_sales_df = home_sales_df.withColumn("bedrooms",home_sales_df["bedrooms"].cast(IntegerType()))
home_sales_df = home_sales_df.withColumn("bathrooms",home_sales_df["bathrooms"].cast(IntegerType()))
home_sales_df = home_sales_df.withColumn("sqft_living",home_sales_df["sqft_living"].cast(IntegerType()))
home_sales_df = home_sales_df.withColumn("sqft_lot",home_sales_df["sqft_lot"].cast(IntegerType()))
home_sales_df = home_sales_df.withColumn("floors",home_sales_df["floors"].cast(IntegerType()))

# Converting relavant column to appropriate date/time datatypes
home_sales_df = home_sales_df.withColumn("date_built",year(home_sales_df["date_built"]))


In [16]:
# Reviewing Schema again
home_sales_df.printSchema()


root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [19]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT date_built, cast(AVG(price) as numeric (9,2)) as avgPrice
FROM home_sales
WHERE bedrooms==4
GROUP BY date_built
ORDER BY date_built DESC
"""
).show()


+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2017|296576.69|
|      2016|296050.24|
|      2015|307908.86|
|      2014|299073.89|
|      2013|299999.39|
|      2012|298233.42|
|      2011|302141.90|
|      2010|296800.75|
+----------+---------+



In [21]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built, cast(AVG(price) as numeric (9,2)) as avgPrice
FROM home_sales
WHERE bedrooms==3
AND bathrooms==3
GROUP BY date_built
ORDER BY date_built DESC
"""
).show()



+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015|288770.30|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [23]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT date_built, cast(AVG(price) as numeric (9,2)) as avgPrice
FROM home_sales
WHERE bedrooms==3
AND bathrooms==3
AND floors == 2
AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built DESC
"""
).show()



+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2017|280317.58|
|      2016|293965.10|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [28]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, cast(AVG(price) as numeric (9,2)) as avgPrice
FROM home_sales
GROUP BY view
HAVING avgPrice >= 350000
order by view DESC
"""
).show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95|1054325.60|
|  94|1033536.20|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87|1072285.20|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82|1063498.00|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 1.3826868534088135 seconds ---


In [29]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [30]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [31]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, cast(AVG(price) as numeric (9,2)) as avgPrice
FROM home_sales
GROUP BY view
HAVING avgPrice >= 350000
order by view DESC
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95|1054325.60|
|  94|1033536.20|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87|1072285.20|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82|1063498.00|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.7119483947753906 seconds ---


In [32]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [33]:
# 11. Read the parquet formatted data.
p_home_sales_df = spark.read.parquet('home_sales')

In [34]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('p_home_sales')

In [35]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
spark.sql("""
SELECT view, cast(AVG(price) as numeric (9,2)) as avgPrice
FROM p_home_sales
GROUP BY view
HAVING avgPrice >= 350000
order by view DESC
"""
).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95|1054325.60|
|  94|1033536.20|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87|1072285.20|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82|1063498.00|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 1.3047196865081787 seconds ---


In [36]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [37]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False